In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time
import os
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from tqdm.notebook import tqdm_notebook

In [3]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
tqdm_notebook.pandas()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/sasipim/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [4]:
PATH = './Surat/test/surat_test.csv'
SAVE_PATH = './Surat/test/surat_test_wind_temp.csv'
LAT = '9.126057'
LONG = '99.325355'

In [5]:
data = pd.read_csv(PATH)
data = data.rename(columns={"Unnamed: 0": "date"})
data.head()

,date,PM2.5
0,2020-07-01 00:00:00,19.0
1,2020-07-01 01:00:00,22.0
2,2020-07-01 02:00:00,28.0
3,2020-07-01 03:00:00,25.0
4,2020-07-01 04:00:00,19.0


In [6]:
data['date_utc'] = pd.to_datetime(data['date']) - timedelta(hours=7)
data['url_date'] = data['date_utc'].apply(lambda x : x.strftime("%Y/%m/%d/%H%M")+"Z")

In [16]:
def getTemp(lat,long,date):
    url = "https://earth.nullschool.net/#"+date+'/wind/surface/level/overlay=temp/orthographic=97.73,-347.68,3906/loc='+ long+ ','+lat
    driver.get(url=url)
    time.sleep(2)
    html = driver.execute_script("return document.documentElement.outerHTML")
    # element = WebDriverWait(driver, 20).until(lambda x: x.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card'))
    all_link = driver.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card')
    for link in all_link:
        return link.text.split("\n")[-2]

In [17]:
def getWind(lat,long,date):
    url = "https://earth.nullschool.net/#"+date+'/wind/isobaric/850hPa/orthographic=97.73,-347.68,3906/loc='+ long+ ','+lat
    driver.get(url=url)
    time.sleep(2)
    html = driver.execute_script("return document.documentElement.outerHTML")
    # element = WebDriverWait(driver, 20).until(lambda x: x.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card'))
    all_link = driver.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card')
    for link in all_link:
        return link.text.split("\n")[-2]

In [19]:
data['temp'] = data['url_date'].progress_apply(lambda x : getTemp(LAT,LONG,x))

  0%|          | 0/8784 [00:00<?, ?it/s]

In [20]:
data['wind']=data['url_date'].progress_apply(lambda x : getWind(LAT,LONG,x))

  0%|          | 0/8784 [00:00<?, ?it/s]

In [21]:
data.to_csv(SAVE_PATH,index=False)